In [1]:
import os
import pandas as pd

In [2]:
genes = pd.read_csv("../../result/network_GTEx_phastcons/valid_genes", sep="\t")
id2genes = genes.set_index("ID")["genes"].to_dict()
genes2id = genes.set_index("genes")["ID"].to_dict()
genes = genes['genes'].values

### download all BED files

In [ ]:
meta = pd.read_csv("../../data/TIP/metadata.tsv", sep="\t")
meta['TF'] = meta['Experiment target'].str.split("-human", expand=True)[0]

data = sc.read_h5ad("../../data/perturb/K562_gwps_normalized_bulk_01.h5ad")
name2id = data.var["gene_name"].to_frame().reset_index().set_index("gene_name")["gene_id"].to_dict()
meta['TFname'] = meta['TF'].map(name2id)

meta = meta[meta['TFname'].isin(genes)] # keep TF in genes only
meta = meta[meta['Output type'] == "IDR thresholded peaks"]
meta.iloc[:, [0, -1]].to_csv("../../data/TIP/meta_GTEx.tsv", sep="\t", index=False)

In [ ]:
with open("joblist", "w") as ff:
    for fid in meta['File accession'].values:
        if not os.path.exists("../../data/TIP/ENCODE/%s.bed.gz" % fid):
            ff.write("wget https://www.encodeproject.org/files/%s/@@download/%s.bed.gz\n" % (fid, fid))
# downloaded files are moved to ../../data/TIP/ENCODE/

### run TIP

In [ ]:
meta = pd.read_csv("../../data/TIP/meta_GTEx.tsv", sep="\t")
with open("joblist", "w") as f:
    for fid in meta["File accession"]:
        f.write("module load R; Rscript TIP.R ../../data/TIP/gene_annotation.txt ../../data/TIP/ENCODE/%s.bed.gz ../../data/TIP/weight/%s.txt ../../data/TIP/score/%s.txt\n" % (fid, fid, fid))

### get_qvalue

In [ ]:
meta = pd.read_csv("../../data/TIP/meta_GTEx.tsv", sep="\t")
with open("joblist", "w") as f:
    for fid in meta["File accession"]:
        f.write("module load R; Rscript TIP_qval.R ../../data/TIP/score/%s.txt ../../data/TIP/qval/%s.txt\n" % (fid, fid))

### parse result to network

In [3]:
meta = pd.read_csv("../../data/TIP/meta_GTEx.tsv", sep="\t")

In [4]:
result = {}
for fid, TF in meta.values:
    df = pd.read_csv("../../data/TIP/qval/%s.txt" % fid, sep="\t")
    targets = set(df['name'].unique())
    if TF in result.keys():
        result[TF] = set.union(result[TF], targets)
    else:
        result[TF] = targets

In [6]:
source = []
target = []
for ii in result.keys():
    source += [ii] * len(result[ii])
    target += list(result[ii])

In [7]:
net = pd.DataFrame([source, target]).T

net[0] = net[0].map(genes2id)
net[1] = net[1].map(genes2id)

net = net.dropna().copy() # remove any NA
net[0] = net[0].astype(int)
net[1] = net[1].astype(int)

In [8]:
net = net.drop_duplicates().sort_values([0, 1])
net.to_csv("../../result/network_GTEx_phastcons/ChIP_TIP.tsv", sep="\t", header=False, index=False)